In [30]:
import urllib.request
import bs4
from bs4 import BeautifulSoup
import re
import os
import json
from subprocess import check_output
import pandas as pd
from datetime import datetime

In [31]:
def timestampToHuman(date):
    return datetime.utcfromtimestamp(date / 1000).strftime('%d%m%Y_%H%M%S')


def getTimeStampFromData(modelJSON):
    date = modelJSON[0]["DSServices"]["date"]
    return date


def downloadTeslaModelPage(url):
    response = urllib.request.urlopen(url)
    html = response.read().decode()
    soup = bs4.BeautifulSoup(html)
    return soup


def buildIDForJSONFile(model, locale, downloadDate):
    return f"{model}_{locale}_{downloadDate}"


def getModel(modelJSON):
    return "modely"


def getLocale(modelJSON):
    return modelJSON[1]["App"]["locale"]


def getDownloadDate(jsonData):
    return timestampToHuman(getTimeStampFromData(jsonData))


def getExportData(jsonData):
    return {
        "model": getModel(jsonData),
        "locale": getLocale(jsonData),
        "downloadDate": getDownloadDate(jsonData)
    }


def deleteTempJavascriptFile(tempFile):
    os.remove(tempFile)

    
def getTeslaModelJSON(soup_page):
    script = soup_page.find('script', text=re.compile(r'dataJson'))
    matches = re.finditer(r'const\s*\w*\s*=\s*\{.+?\};', script.text, re.DOTALL)
    dict_matches = [re.sub(r'const\s*\w*\s*=\s*', '', match.group(0)).rstrip(";") for match in matches]
    concat_dicts = ",".join(dict_matches)
    js_export_json_string = 'const teslaData = [' + concat_dicts + '];\nprocess.stdout.write(JSON.stringify(teslaData));'
    with open('temp.js', 'w') as f:
        f.write(js_export_json_string)
    modelData = json.loads(check_output(['node','temp.js']).decode())
    deleteTempJavascriptFile("temp.js")

    
    exportData = getExportData(modelData)
    exportID = buildIDForJSONFile(exportData["model"], exportData["locale"], exportData["downloadDate"])
    exportFilename = "raw_" + exportID + '.json'
    
    exportJSONToFile(data_dir + exportFilename, modelData)
    return exportFilename


def exportJSONToFile(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f)


def importModelJSON(json_file):
    with open(json_file, 'r') as f:
        return f.read()

In [32]:
models = ["modely", "model3", "models", "modelx"]
country_codes = []
data_dir = "data/"

In [33]:
url = "https://www.tesla.com/modely/design#overview"

In [34]:
modely_page = downloadTeslaModelPage(url)

In [35]:
filename = getTeslaModelJSON(modely_page)

### Loading JSON Data!

In [36]:
modelyJSON = json.loads(importModelJSON(data_dir + filename))

In [37]:
availableLocales = modelyJSON[1]["App"]["availableLocales"]

In [38]:
# LOCALE SELECTOR!
localeSelectorJSON = json.loads(modelyJSON[1]["App"]["localeSelector"])
regionsList = localeSelectorJSON["localeSelector"]

def getCountryCode(regionsList):
    country_list = []
    for region in regionsList:
        continent = region["regions"]
        for continentList in continent:
            countries = continentList["countries"]
            for country in countries:
                countryName = country["countryName"]
                locale = country["languages"][0]["localeCode"] 
                country_list.append({
                    "country": countryName,
                    "locale": locale
                })
    
    return country_list

country_codes = getCountryCode(regionsList)

## Get Vehicle Information
- Get base price for "Model Y Long Range" (65990)
- Get Est. Delivery in same format as on webpage (Jan - Mar 2023)
- Get available information: Range, Top Speed, Acceleration
- Get all available trims (Long Range + Performance
)
- Get all Options with prices: (color, wheels, tow hitch, interior, seats, enhaced Autopilot, FSD, charging: wall connector + mobile connector)

## 1. Data MVP
```
first_mvp = {
	"model": "modely",
	"price": {
		"currency": "USD",
		"amount": 69420
	},
	"deliveryEstimate": {
		"start": ,
		"end": ,
		"text": "1 - 8 weeks"
	}
}
```

## 2. Data MVP

Missing: 
- locale!


```
second_mvp = {
    "model": "Model Y",
    "date": 1673565825608,
    "meta": {
        "currency": "USD",
        "symbol": "$",
    },
	"trim": {
        "base": {
            "price": 69420
            "deliveryEstimate": {
                "start": ,
                "end": ,
                "text": "1 - 8 weeks"
            }
        },
        "performance": {
            "price": 69420,
            "deliveryEstimate": {
                "start": ,
                "end": ,
                "text": "1 - 8 weeks"
            }
        } 
    }
}
```

In [39]:
second_mvp = {
    "model": "Model Y",
    "date": 1673565825608,
    "meta": {
        "currency": "USD",
        "symbol": "$",
        "range_units": "miles",
        "range_source": "EPA"
    },
	"trim": {
        "base": {
            "price": 69420,
            "deliveryEstimate": {
                "start": 123,
                "end": 234,
                "text": "1 - 8 weeks"
            },
            "options": {},
            "specs": {},
        },
        "performance": {
            "price": 69420,
            "deliveryEstimate": {
                "start": 123,
                "end": 234,
                "text": "1 - 8 weeks"
            }
        } 
    }
}

In [40]:
def dateStringToMonth(datestring, date_format='%m/%d/%Y %H:%M:%S'):
    date = datetime.strptime(datestring, date_format)
    year = date.year
    month_string = date.strftime('%B')
    return month_string, year


def estDeliveryString(start, end, date_format='%m/%d/%Y %H:%M:%S'):
    """Takes the start and end date in the format '%m/%d/%Y %H:%M:%S' and returns the string in the formart 
    Month - Month YEAR.
    If the Years are different, it will return
    Month YEAR - Month YEAR"""
    start = dateStringToMonth(start)
    end = dateStringToMonth(end)
    if start[1] == end[1]:
        date_string = f'{start[0]} - {end[0]} {end[1]}'
    else:
        date_string = f'{start[0]} {start[1]} - {end[0]} {end[1]}'
    
    return date_string


def getDeliveryEstForTrim(trim, modelJSON):
    eD = eddDataForModel(trim, modelyJSON)
    deliveryEstForTrim = deliveryEstimateForModel(eD)
    eddString = estDeliveryString(deliveryEstForTrim["start"], deliveryEstForTrim["end"])
    
    deliveryEstimate = {
        "start": deliveryEstForTrim["start"],
        "end": deliveryEstForTrim["end"],
        "text": eddString
    }
    
    return deliveryEstimate

In [41]:
def getAllTrims(modelJSON):
    trims = modelyJSON[0]["DSServices"]["Lexicon.my"]["sku"]["trims"]
    return trims

def getConfigurableTrims(modelJSON):
    """Scrapes the JSON scraped from the Tesla webpage and returns a dictionary of all configurable trims with corresponding options"""
    all_trims = getAllTrims(modelJSON)
    configurable_trims = {k: v for k, v in all_trims.items() if "configurator" in v}
    return configurable_trims


def explainConfigurableTrims(configurable_trims):
    """Takes the trims dictionary and returns a dictionary with the shortcut and the corresponding name"""
    new_dict = {}
    for key, value in configurable_trims.items():
        new_dict[key] = value["variant"]["name"]
    
    return new_dict

def explainTrim(trim):
    """Takes the shortcut of one trim and returns the name/description of the trim"""
    try:
        return configurable_trims[trim]["variant"]["name"]
    except:
        print("There is no trim available with the shortcut " + trim)

In [42]:
def getDate(modelJSON):
    """Returns the time of data download as Milliseconds EPOCH"""
    return modelJSON[0]["DSServices"]["date"]


def getModel(modelJSON):
    """Returns the Model Name as shortcut like my for ModelY or m3 for Model3"""
    return modelJSON[0]["DSServices"]["KeyManager"]["keys"]["Lexicon"][0]["query"]["model"]


def getLexicon(modelJSON):
    """Returns the Lxicon for given modelJSON"""
    lexiconKey = modelJSON[0]["DSServices"]["KeyManager"]["keys"]["Lexicon"][0]["key"]
    return modelJSON[0]["DSServices"][lexiconKey]


def priceForOption(option, modelJSON):
    """Receives the option as '$MTY05' and the modelJSON and returns the price for that option"""
    lexicon = getLexicon(modelJSON)
    
    price = lexicon["options"][option]["price"]
    for jsonObj in lexicon["options"][option]["pricing"]:
        if jsonObj["type"] == "base_plus_trim":
            price = jsonObj["value"]
    
    return price


def nameForOption(option, modelJSON):
    """Receives the option in '$MTY05' format and the modelJSON, return the name of the option"""
    lexicon = getLexicon(modelJSON)
    
    name = lexicon["options"][option]["long_name"]
    return name


def eddDataForModel(model, modelJSON):
    """Returns the estimated Delivery Data for given model
    Inputs:
    - model = '$MTY05'
    - modelJSON = the Model JSON object
    
    Returns:
    List with estDeliveryData for given model
    """
    
    edd = modelJSON[1]["eddData"]
    eddForModel = []
    
    for option in edd:
        if model in option["options"]:
            eddForModel.append(option)
    
    return eddForModel


def getValidEddData(eddDataForModel):
    """Throws away the inStart and inEnd dates that have type None.
    Returns a list of valid est Delivery Data"""
    
    validEddData = []
    for key, value in enumerate(eddDataForModel):
        if value["inStart"] is not None and value["inEnd"] is not None:
            validEddData.append(value)
            
    return validEddData


def deliveryEstimateForModel(eddDataForModel):
    """takes the estDelivery Data  as a list of objects 
    and returns the start, end and outliers as dict"""
    validEddData = getValidEddData(eddDataForModel)
    start = eddDataForModel[0]["inStart"]
    end = eddDataForModel[0]["inEnd"]
    
    deliveryEst = {
        "start": start,
        "end": end,
        "outliers": []
    }
        
    for obj in validEddData:
        if obj["inStart"] != start and obj["inEnd"] != end:
            deliveryEst["outliers"].append(obj)
    return deliveryEst


def getMetaData(modelJSON):
    lexicon = getLexicon(modelJSON)
    meta = lexicon["metadata"]
    metaData = {
        "country": modelJSON[1]["App"]["uiCountry"],
        "currency": meta["currency_code"],
        "symbol": meta["currency_symbol"],
        "range_units": meta["specs"]["data"][0]["meta"]["specs"]["range"]["units"],
        "range_source": meta["specs"]["data"][0]["meta"]["specs"]["range"]["source"]
    }
    return metaData



def getTrimInfo(trim, modelJSON):
    """Returns the Trim Info for given trim
    Input:
    - trim = '$MTY05'
    - modelJSON = The model JSON
    
    Returns:
    - trimName as String
    - trimDict as dict with price, delivery est."""
    
    trimName = nameForOption(trim, modelJSON)
    trimDict = {
        "price": priceForOption(trim, modelJSON),
        "deliveryEstimate": getDeliveryEstForTrim(trim, modelJSON),
        "specs": getSpecsForTrim(trim, modelJSON),
        "options": "options"
    }
    
    
    return trimName, trimDict


def allTrimsInfo(modelJSON):
    trims = {}
    configurable_trims = getConfigurableTrims(modelJSON)
    trimsList = list(configurable_trims.keys())
    for trim in trimsList:
        trimName, trimData = getTrimInfo(trim, modelJSON)
        trims[trimName] = trimData
    
    return trims


def getModelData(modelJSON):
    modelData = {
        "model": getModel(modelyJSON),
        "date": getDate(modelJSON),
        "meta": getMetaData(modelJSON),
        "trim": allTrimsInfo(modelJSON)
    }
    return modelData

In [43]:
def getOptionPriceAndName(option, modelJSON):
    """For a given option returns the dicitonary
    {
    name: "White Interior",
    price: 1200,
    currency: USD,
    }
    """
    lexicon = getLexicon(modelJSON)
    
    price = lexicon["options"][option]["price"]
    for jsonObj in lexicon["options"][option]["pricing"]:
        if jsonObj["type"] == "base_plus_trim":
            price = jsonObj["value"]
    
    optionData = {
        "name": lexicon["options"][option]["name"],
        "price": price,
    }
    
    
    return optionData


In [44]:
def getSpecsForTrim(trim, modelJSON):
    """Takes the trim and modelJSON and returns a list
    with the specs: 
    - range
    - max-speed
    - acceleartion
    """
    lexicon = getLexicon(modelJSON)
    trimSpecs = lexicon["metadata"]["specs"]["data"][0]["options"][trim]
    specs = {
        "range": trimSpecs["range"],
        "topspeed": trimSpecs["topspeed"],
        "acceleration": trimSpecs["acceleration"],
        "overrides": specChangingOptions(trimSpecs)
    }
    
    return specs


def specChangingOptions(specs):
    overrides = specs["overrides"]
    for override in overrides:
        for key, option in enumerate(override['selected_by']['and']):
            if option.startswith('$'):
                override['selected_by']['and'][key] = getOptionPriceAndName(option, modelyJSON)
    return overrides


def optionsCodeWithExplanation(modelJSON):
    options = modelyJSON[0]["DSServices"]["Lexicon.my"]["options"]
    new_dict = {}
    for key, value in options.items():
        new_dict[key] = value["name"]
        
    return new_dict


def explainThisOption(option):
    """Returns the name/description of the prompted option"""
    explanation = ""
    if "$" in option:
        explanation = explainedOptions[option]
    else:
        explanation = explainedOptions["$" + option]
    return explanation


def getOptionPriceAndName(option, modelJSON):
    """For a given option returns the dicitonary
    {
        shortcut: $MTY05
        name: "White Interior",
        price: 1200,
    }
    """
    lexicon = getLexicon(modelJSON)
    
    price = lexicon["options"][option]["price"]
    for jsonObj in lexicon["options"][option]["pricing"]:
        if jsonObj["type"] == "base_plus_trim":
            price = jsonObj["value"]
    
    optionData = {
        "shortcut": option,
        "name": lexicon["options"][option]["name"],
        "price": price,
    }
    
    
    return optionData

In [27]:
explainedOptions = optionsCodeWithExplanation(modelyJSON)

In [45]:
getModelData(modelyJSON)

{'model': 'my',
 'date': 1674033695084,
 'meta': {'country': 'US',
  'currency': 'USD',
  'symbol': '$',
  'range_units': 'miles',
  'range_source': 'EPA'},
 'trim': {'Model Y Performance Dual Motor All-Wheel Drive': {'price': 56990,
   'deliveryEstimate': {'start': '01/01/2023 00:00:00',
    'end': '02/01/2023 00:00:00',
    'text': 'January - February 2023'},
   'specs': {'range': 303,
    'topspeed': 155,
    'acceleration': 3.5,
    'overrides': [{'content': {'range': 303,
       'topspeed': 155,
       'acceleration': 3.5},
      'selected_by': {'and': [{'shortcut': '$SPTY1',
         'name': 'Performance Upgrade',
         'price': 0}]}},
     {'content': {'range': 303},
      'selected_by': {'and': [{'shortcut': '$STY5S',
         'name': 'Five Seat Interior',
         'price': 0}]}}]},
   'options': 'options'},
  'Model Y Long Range Dual Motor All-Wheel Drive': {'price': 52990,
   'deliveryEstimate': {'start': '01/01/2023 00:00:00',
    'end': '03/01/2023 00:00:00',
    'text':

## Get Est Delivery Data for Model

In [59]:
mty05_edd = eddDataForModel("$MTY05", modelyJSON)

In [14]:
configurable_trims = getConfigurableTrims(modelyJSON)

# Make Model JSON with names and prices
```
{
    name: "White Interior",
    price: 1200,
    currency: USD,
}
```

In [67]:
def getOptionPriceAndName(option, modelJSON):
    """For a given option returns the dicitonary
    {
    name: "White Interior",
    price: 1200,
    currency: USD,
    }
    """
    lexicon = getLexicon(modelJSON)
    
    price = lexicon["options"][option]["price"]
    for jsonObj in lexicon["options"][option]["pricing"]:
        if jsonObj["type"] == "base_plus_trim":
            price = jsonObj["value"]
    
    optionData = {
        "shortcut": option,
        "name": lexicon["options"][option]["name"],
        "price": price,
    }
    
    
    return optionData

In [68]:
getOptionPriceAndName("$STY7S", modelyJSON)

{'shortcut': '$STY7S', 'name': 'Seven Seat Interior', 'price': 4000}

In [66]:
{
    "trim": list(configurable_trims.keys())[0],
    "configurator": configurable_trims["$MTY05"]["configurator"][0]
}

{'trim': '$MTY05',
 'configurator': {'default': False,
  'optional': [['$APF2', '$APPB'], ['$TW01']],
  'upgradable': [],
  'base_options': ['$MTY05',
   '$DV4W',
   '$SPTY1',
   '$BCYR',
   '$PL31',
   '$SLR1',
   '$WY21P',
   '$MDLY',
   '$STY5S',
   '$CPF0',
   '$APBS'],
  'combinations': [['$PPSW', '$PMNG', '$PPSB', '$PBSB', '$PPMR'],
   ['$INPB0', '$INPW0'],
   ['$SC04', '$SC05', '$SC06']]}}

In [67]:
base_options = configurable_trims["$MTY05"]["configurator"][0]["base_options"]
for option in base_options:
    print(getOptionPriceAndName(option, modelyJSON))

GOING IN FOR OPTION:  $MTY05
{'name': 'Performance All-Wheel Drive', 'price': 69990, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $DV4W
{'name': 'Dual Motor All-Wheel Drive', 'price': 0, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $SPTY1
{'name': 'Performance Upgrade', 'price': 0, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $BCYR
{'name': 'Performance Brakes', 'price': 0, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $PL31
{'name': 'Performance Pedals', 'price': 0, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $SLR1
{'name': 'Carbon Fiber Spoiler', 'price': 0, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $WY21P
{'name': '21’’ Überturbine Wheels', 'price': 0, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $MDLY
{'name': 'Model Y', 'price': 39990, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $STY5S
{'name': 'Five Seat Interior', 'price': 0,

In [15]:
configurator = configurable_trims["$MTY05"]["configurator"][0]

In [114]:
configurator

{'default': False,
 'optional': [[{'name': 'Full Self-Driving Capability',
    'price': 15000,
    'currency': 'USD',
    'currencySymbol': '$'},
   {'name': 'Enhanced Autopilot',
    'price': 6000,
    'currency': 'USD',
    'currencySymbol': '$'}],
  [{'name': 'Tow Hitch',
    'price': 1000,
    'currency': 'USD',
    'currencySymbol': '$'}]],
 'upgradable': [],
 'base_options': ['$MTY05',
  '$DV4W',
  '$SPTY1',
  '$BCYR',
  '$PL31',
  '$SLR1',
  '$WY21P',
  '$MDLY',
  '$STY5S',
  '$CPF0',
  '$APBS'],
 'combinations': [[{'name': 'Pearl White Multi-Coat',
    'price': 0,
    'currency': 'USD',
    'currencySymbol': '$'},
   {'name': 'Midnight Silver Metallic',
    'price': 0,
    'currency': 'USD',
    'currencySymbol': '$'},
   {'name': 'Deep Blue Metallic',
    'price': 1000,
    'currency': 'USD',
    'currencySymbol': '$'},
   {'name': 'Solid Black',
    'price': 1500,
    'currency': 'USD',
    'currencySymbol': '$'},
   {'name': 'Red Multi-Coat',
    'price': 2000,
    'currency

In [91]:
for option in configurator["optional"]:
    if type(option) == list:
        for key, item in enumerate(option):
            print(getOptionPriceAndName(item, modelyJSON))
            #option[key] = getOptionPriceAndName(item, modelyJSON)


GOING IN FOR OPTION:  $APF2
{'name': 'Full Self-Driving Capability', 'price': 15000, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $APPB
{'name': 'Enhanced Autopilot', 'price': 6000, 'currency': 'USD', 'currencySymbol': '$'}
GOING IN FOR OPTION:  $TW01
{'name': 'Tow Hitch', 'price': 1000, 'currency': 'USD', 'currencySymbol': '$'}


In [ ]:
print(getOptionPriceAndName(item, modelyJSON))
    
        for key, item in enumerate(optionsList):
            print("item in OPTIONSLIST", item)
            print(getOptionPriceAndName(item, modelyJSON))
            if type(optionsList) == list:
                print("if List", item)
                for key, item in enumerate(optionsList):
                    print(getOptionPriceAndName(item, modelyJSON))

In [12]:
def getConfigForAllTrims(configurableTrims):
    trimConfigs = {}
    for trim in list(configurableTrims.keys()):
        configurator = configurable_trims[trim]["configurator"][0]
        trimConfig = getConfiguratorInfo(configurator)
        trimConfigs[trim] = trimConfig
        
    return trimConfigs
    
    
def getConfiguratorInfo(configurator):
    infoConfigurator = configurator
    configKeysList = ["optional", "base_options", "combinations"]
    for key in configKeysList:
        print("CONFIGRATOR[KEY]", infoConfigurator[key])
        tempOptionsList = []
        for i, element in enumerate(infoConfigurator[key]):
            if type(element) == str:
                infoConfigurator[key][i] = getOptionPriceAndName(element, modelyJSON)
            else:
                for key, item in enumerate(element):
                    try:
                        element[key] = getOptionPriceAndName(item, modelyJSON)
                    except KeyError:
                        print(item, "is not a valid option!")
    return infoConfigurator

In [30]:
trimConfig = getConfigForAllTrims(configurable_trims)

CONFIGRATOR[KEY] [['$APF2', '$APPB'], ['$TW01']]
NOT A STRING ['$APF2', '$APPB']
GOING IN FOR OPTION:  $APF2
NOT A STRING [{'name': 'Full Self-Driving Capability', 'price': 15000, 'currency': 'USD', 'currencySymbol': '$'}, '$APPB']
GOING IN FOR OPTION:  $APPB
NOT A STRING ['$TW01']
GOING IN FOR OPTION:  $TW01
CONFIGRATOR[KEY] ['$MTY05', '$DV4W', '$SPTY1', '$BCYR', '$PL31', '$SLR1', '$WY21P', '$MDLY', '$STY5S', '$CPF0', '$APBS']
['$MTY05', '$DV4W', '$SPTY1', '$BCYR', '$PL31', '$SLR1', '$WY21P', '$MDLY', '$STY5S', '$CPF0', '$APBS']
GOING IN FOR OPTION:  $MTY05
ELEMENT::  $MTY05
[{'name': 'Performance All-Wheel Drive', 'price': 69990, 'currency': 'USD', 'currencySymbol': '$'}, '$DV4W', '$SPTY1', '$BCYR', '$PL31', '$SLR1', '$WY21P', '$MDLY', '$STY5S', '$CPF0', '$APBS']
GOING IN FOR OPTION:  $DV4W
ELEMENT::  $DV4W
[{'name': 'Performance All-Wheel Drive', 'price': 69990, 'currency': 'USD', 'currencySymbol': '$'}, {'name': 'Dual Motor All-Wheel Drive', 'price': 0, 'currency': 'USD', 'currency

## OPTIONS of the Model

In [41]:
# All available options
def getModelOptions(modelJSON):
    return modelJSON[0]["DSServices"]["Lexicon.my"]["options"]

options = getModelOptions(modelyJSON)


def getPricedOptions(options):
    """Takes the options object/dict as input and return a dictionary with the options and their correpsonding pricing"""
    new_dict = {}
    important_values = ["name", "price", "pricing"]
    result = {key: {k: v for k, v in value.items() if k in important_values} for key, value in options.items()}
    return result

pricedOptions = getPricedOptions(options)

In [40]:
explainedOptions

{'$APBS': 'Basic Autopilot',
 '$APF2': 'Full Self-Driving Capability',
 '$APPB': 'Enhanced Autopilot',
 '$BCYR': 'Performance Brakes',
 '$CPF0': 'Standard Connectivity',
 '$CPF1': 'Premium Connectivity',
 '$DV2W': 'Rear-Wheel Drive',
 '$DV4W': 'Dual Motor All-Wheel Drive',
 '$FM3U': 'Acceleration Boost',
 '$MDLY': 'Model Y',
 '$PBSB': 'Solid Black',
 '$PL31': 'Performance Pedals',
 '$PMNG': 'Midnight Silver Metallic',
 '$PPMR': 'Red Multi-Coat',
 '$PPSB': 'Deep Blue Metallic',
 '$PPSW': 'Pearl White Multi-Coat',
 '$SC04': 'Supercharger Network Access + Pay-as-you-go',
 '$SLR1': 'Carbon Fiber Spoiler',
 '$TW01': 'Tow Hitch',
 '$INPB0': 'All Black Premium Interior',
 '$INPW0': 'Black and White Premium Interior',
 '$MTY05': 'Performance All-Wheel Drive',
 '$MTY11': 'Long Range All-Wheel Drive',
 '$PRMY1': 'Premium Interior',
 '$SPTY1': 'Performance Upgrade',
 '$STY5S': 'Five Seat Interior',
 '$STY7S': 'Seven Seat Interior',
 '$WY19B': '19’’ Gemini Wheels',
 '$WY20P': '20’’ Induction Wheel

In [24]:
explainThisOption("STY5S")

'Five Seat Interior'

### Get base price for "Model Y Long Range" (65990)


In [20]:
def getPriceForTrim(trim):
    """Takes in the options JSON and the trim shortcut in teh format '$XXXX' and returns the price for that trim"""
    price = 0
    for jsonObj in options[trim]["pricing"]:
        if jsonObj["type"] == "base_plus_trim":
            price = jsonObj["value"]
    return price

getPriceForTrim("$MTY05")

69990

In [21]:
def pricesForTrims(available_trims):
    """Takes in the list of available trims and returns a dict with country code, model, trim, price and currency"""
    trim_price_infos = []
    for trim in available_trims:
        for jsonObj in options[trim]["pricing"]:
            if jsonObj["type"] == "base_plus_trim":
                trim_price_infos.append({
                    "country code": lexicon["market"],
                    "model": lexicon["product"],
                    "trim": explainTrim(trim),
                    "shortcut": trim,
                    "price": jsonObj["value"],
                    "currency": jsonObj["context"]
                })
    return trim_price_infos

pricesForTrims(available_trims)

NameError: name 'available_trims' is not defined

### Get Est. Delivery in same format as on webpage (Jan - Mar 2023)

In [76]:
def get_aggregated_delivery_estimates(delivery_estimates):
    temp = {}
    temp_list = []
            
    for delivery_estimate in delivery_estimates:
        country_code = delivery_estimate['countryCode']
        region = delivery_estimate['region']
        in_start = delivery_estimate['inStart']
        in_end = delivery_estimate['inEnd']

        if "inStart" not in temp:
            temp["inStart"] = in_start
            temp["inEnd"] = in_end
            temp["region"] = region

        
        if in_start != temp["inStart"]:
            temp_list.append(v)
    
    if len(temp_list) == 0:
        return temp
            
    return temp_list, temp

aggregated_delivery_estiamtes = get_aggregated_delivery_estimates(estDeliveryDataModelY)
estDeliveryString(aggregated_delivery_estiamtes["inStart"], aggregated_delivery_estiamtes["inEnd"])

'January - March 2023'

## Get available information: Range, Top Speed, Acceleration

Range: 0 -- DSServices -- Lexicon.my -- metadata -- specs -- data -- 0 -- options -- $MTY11 -- (overrides [0-4])

Units: `modelyJSON[0]["DSServices"]["Lexicon.my"]["metadata"]["specs"]["data"][0]["meta"]["specs"]`

In [120]:
def getTopSpeedForTrim(trim="$MTY11"):
    return modelyJSON[0]["DSServices"]["Lexicon.my"]["metadata"]["specs"]["data"][0]["options"][trim]["topspeed"]

getTopSpeedForTrim("$MTY11")

135

In [118]:
def getAccelerationForTrim(trim="$MTY11"):
    return modelyJSON[0]["DSServices"]["Lexicon.my"]["metadata"]["specs"]["data"][0]["options"][trim]["acceleration"]

getAccelerationForTrim("$MTY11")

4.8

In [117]:
def getRangeForTrim(trim="$MTY11"):
    return modelyJSON[0]["DSServices"]["Lexicon.my"]["metadata"]["specs"]["data"][0]["options"][trim]["range"]

getRangeForTrim("$MTY11")

330

## Get all Options with prices: (color, wheels, tow hitch, interior, seats, enhaced Autopilot, FSD, charging: wall connector + mobile connector)

Get all available options for the given trim

In [19]:
def sub_values(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, str) and value[0] == "$":
                data[key] = explainedOptions[value]
            else:
                sub_values(value)
    elif isinstance(data, list):
        for i, item in enumerate(data):
            sub_values(item)

    return data
substitute_values(mty05)


NameError: name 'substitute_values' is not defined

In [35]:
def substitute_values(mty05, explainedOptions):
    for key in mty05:
        print(key)
        if isinstance(mty05[key], list):
            print("islist", mty05, "\n")
            for index, item in enumerate(mty05[key]):
                print(index, item, "\n")
                if isinstance(item, dict):
                    print("substitute again", item, "\n")
                    substitute_values(item, explainedOptions)
                elif isinstance(item, str) and item[0] == "$":
                    print("isString", item, mty05[index], "\n")
                    mty05[index] = explainedOptions[item]
        elif isinstance(mty05[key], dict):
            print("Substitute! ", key, mty05[key], "\n")
            substitute_values(mty05[key], explainedOptions)
        elif isinstance(mty05[key], str) and mty05[key][0] == "$":
            mty05[key] = explainedOptions[mty05[key]]
            
    return mty05


In [38]:
pricedOptions

{'$APBS': {'name': 'Basic Autopilot',
  'price': 0,
  'pricing': [{'type': 'trial', 'value': 3000, 'context': 'USD'},
   {'type': 'base', 'value': 0, 'context': 'USD'},
   {'type': 'upgrade', 'value': 3000, 'context': 'USD'}]},
 '$APF2': {'name': 'Full Self-Driving Capability',
  'price': 15000,
  'pricing': [{'type': 'trial', 'value': 15000, 'context': 'USD'},
   {'type': '1_month',
    'value': 199,
    'context': 'USD',
    'set_rules': [{'action': 'set',
      'target': '$APF2',
      'target_value': 99,
      'condition_code': '$APPB',
      'condition_state': 'set',
      'target_property': 'price'}]},
   {'type': 'base', 'value': 15000, 'context': 'USD'},
   {'type': 'upgrade',
    'value': 15000,
    'context': 'USD',
    'set_rules': [{'action': 'set',
      'target': '$APF2',
      'target_value': 9000,
      'condition_code': '$APPB',
      'condition_state': 'set',
      'target_property': 'price'}]}]},
 '$APPB': {'name': 'Enhanced Autopilot',
  'price': 6000,
  'pricing': 

In [109]:
configurator_list = []
for jsonObj in modelyJSON[0]["DSServices"]["Lexicon.my"]["groups"]:
    if jsonObj["context"] == "configurator":
        configurator_list.append(jsonObj)

In [112]:
range_numbers_list = [dic for dic in configurator_list if "extra_content" in dic]

In [113]:
range_numbers_list

[{'code': 'REAR_SEATS',
  'name': 'REAR_SEATS',
  'context': 'configurator',
  'extra_copy': [{'type': 'name', 'content': 'Seating Layout'},
   {'type': 'certified_range', 'content': 'Range (EPA est.) : {VALUE}mi'},
   {'type': 'certified_range_est', 'content': 'Range (est.) : {VALUE}mi'}],
  'extra_content': [{'type': 'certified_disclaimer',
    'content': [{'value': 280,
      'subtype': 'certified_range',
      'selected_by': {'and': ['$WY21P', '$MTY04']}},
     {'value': ' ',
      'subtype': 'certified_range',
      'selected_by': {'and': ['$RANDOM']}},
     {'value': 330,
      'subtype': 'certified_range',
      'selected_by': {'and': ['$WY19B', '$MTY11', '$STY5S']}},
     {'value': 326,
      'subtype': 'certified_range_est',
      'selected_by': {'and': ['$WY19B', '$MTY11', '$STY7S']}},
     {'value': 318,
      'subtype': 'certified_range_est',
      'selected_by': {'and': ['$WY20P', '$MTY11', '$STY5S']}},
     {'value': 314,
      'subtype': 'certified_range_est',
      'sel

In [96]:
explainTrim("$MTY11")

'LONG RANGE ALL-WHEEL DRIVE'

In [20]:
explainThisOption("$WY19B")

NameError: name 'explainedOptions' is not defined

In [100]:
explainThisOption("$STY7S")

'Seven Seat Interior'

In [114]:
explainThisOption("$FM3U")

'Acceleration Boost'

In [36]:
aggregated_delivery_estimates = {}

for delivery_estimate in estDeliveryDataModelY:
    country_code = delivery_estimate['countryCode']
    region = delivery_estimate['region']
    in_start = delivery_estimate['inStart']
    in_end = delivery_estimate['inEnd']

    # Initialize the aggregated delivery estimate for the country if it doesn't exist
    if country_code not in aggregated_delivery_estimates:
        aggregated_delivery_estimates[country_code] = {}

    # Initialize the aggregated delivery estimate for the region if it doesn't exist
    if region not in aggregated_delivery_estimates[country_code]:
        aggregated_delivery_estimates[country_code][region] = {'inStart': in_start, 'inEnd': in_end}
    # Otherwise, update the aggregated delivery estimate if the current delivery estimate has different start or end times
    else:
        aggregated_in_start = aggregated_delivery_estimates[country_code][region]['inStart']
        aggregated_in_end = aggregated_delivery_estimates[country_code][region]['inEnd']
        if in_start != aggregated_in_start or in_end != aggregated_in_end:
            aggregated_delivery_estimates[country_code][region] = {'inStart': in_start, 'inEnd': in_end}


In [78]:
delivery_list = []
for jsonObj in estDeliveryDataModelY:
    deliveryEstimate = {
        "countryCode": jsonObj["countryCode"],
        "region": jsonObj["region"],
        "model": jsonObj["model"],
        "startDelivery": jsonObj["inStart"],
        "endDelivery": jsonObj["inEnd"],
        "estDelivery": estDeliveryString(jsonObj["inStart"], jsonObj["inEnd"]),
        "inType": jsonObj["inType"]
    }
    delivery_list.append(deliveryEstimate)

In [79]:
delivery_list

[{'countryCode': 'US',
  'region': 'west',
  'model': 'my',
  'startDelivery': '01/01/2023 00:00:00',
  'endDelivery': '03/01/2023 00:00:00',
  'estDelivery': 'January - March 2023',
  'inType': 'month_year'},
 {'countryCode': 'US',
  'region': 'texas',
  'model': 'my',
  'startDelivery': '01/01/2023 00:00:00',
  'endDelivery': '03/01/2023 00:00:00',
  'estDelivery': 'January - March 2023',
  'inType': 'month_year'},
 {'countryCode': 'US',
  'region': 'other',
  'model': 'my',
  'startDelivery': '01/01/2023 00:00:00',
  'endDelivery': '03/01/2023 00:00:00',
  'estDelivery': 'January - March 2023',
  'inType': 'month_year'},
 {'countryCode': 'US',
  'region': 'noncontinental(ak/hi)',
  'model': 'my',
  'startDelivery': '01/01/2023 00:00:00',
  'endDelivery': '03/01/2023 00:00:00',
  'estDelivery': 'January - March 2023',
  'inType': 'month_year'},
 {'countryCode': 'US',
  'region': 'noncontinental',
  'model': 'my',
  'startDelivery': '01/01/2023 00:00:00',
  'endDelivery': '03/01/2023 

In [ ]:
def isDeliveryEstSameAcrossRegions(estDeliveryListForTrim):
    """Takes the list of objects for trim estimated deliveries"""
    # takes List for Trim
    # gets inStart + inEnd
    # 

In [45]:
mty = getEstDeliverDataForTrim(trim="$MTY05")

In [46]:
columns=["id", "countryCode", "region", "model", "inStart", "inEnd"]

In [47]:
mty

[{'id': 371870,
  'deliveryWindowDisplay': None,
  'children': [],
  'childIds': [],
  'countryCode': 'US',
  'region': 'west',
  'model': 'my',
  'options': ['$MTY05'],
  'atDate': None,
  'inStart': '01/01/2023 00:00:00',
  'inEnd': '02/01/2023 00:00:00',
  'inType': 'month_year',
  'effectiveStartDate': '2022-12-31T08:00:00'},
 {'id': 371863,
  'deliveryWindowDisplay': None,
  'children': [],
  'childIds': [],
  'countryCode': 'US',
  'region': 'texas',
  'model': 'my',
  'options': ['$MTY05'],
  'atDate': None,
  'inStart': '01/01/2023 00:00:00',
  'inEnd': '02/01/2023 00:00:00',
  'inType': 'month_year',
  'effectiveStartDate': '2022-12-31T08:00:00'},
 {'id': 371884,
  'deliveryWindowDisplay': None,
  'children': [],
  'childIds': [],
  'countryCode': 'US',
  'region': 'other',
  'model': 'my',
  'options': ['$MTY05'],
  'atDate': None,
  'inStart': '01/01/2023 00:00:00',
  'inEnd': '02/01/2023 00:00:00',
  'inType': 'month_year',
  'effectiveStartDate': '2022-12-31T08:00:00'},
 {